In [1]:
import pandas as pd
import os
os.getcwd()


'/Users/marcusmoody/code/Tanguyrhd/vibe/Marcus_notebooks'

In [4]:
df = pd.read_csv('/raw_data/twitter_MBTI.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/raw_data/twitter_MBTI.csv'

In [ ]:
df.columns


In [ ]:
#Turns one row of 50 tweets into 50 rows with one tweet each, but they keep the same MBTI label--400,000 tweet rows instead of 8,000
df['tweets'] = df['text'].apply(lambda x: x.split('|||'))


In [ ]:
df.head()

In [ ]:
df['tweets'][0]

In [ ]:
df['tweets'] = df['tweets'].apply(lambda tweets: [t.strip() for t in tweets])

In [ ]:
df['tweets'][0]

In [ ]:
df_exploded = df.explode('tweets').rename(columns={'tweets': 'tweet'}).reset_index(drop=True)

In [ ]:
df_exploded.head()

In [ ]:
df_exploded

In [ ]:
len(df_exploded)

In [ ]:
df_exploded.duplicated().sum()

In [ ]:
#Clean the data
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www.|S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#\w+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df_exploded['clean_tweet'] = df_exploded['tweet'].apply(clean_text)

df_exploded

In [ ]:
df_exploded.shape

In [ ]:
df_exploded.columns

In [ ]:
df_exploded['tweet'].apply(len).describe()

In [ ]:
df_exploded['clean_tweet'].isnull().sum()

In [ ]:
df_exploded['label'].value_counts()

In [ ]:
df['tweets'].apply(len).describe()

In [ ]:
df_exploded['clean_tweet'][0]

In [ ]:
df['tweets_per_user'] = df['text'].apply(lambda x: len(str(x).split('|||')))
df['tweets_per_user'].describe()

In [ ]:
#reassigning user_id to each tweet based on OG row #
df_exploded['user_id'] = df_exploded['Unnamed: 0']

In [ ]:
#Grouping users
grouped_users = df_exploded.groupby(['user_id', 'label'])['clean_tweet'].apply(lambda tweets: ' '.join(tweets)).reset_index()

In [ ]:
grouped_users.head()

In [ ]:
chunk_data = []

for _, row in grouped_users.iterrows():
    label = row['label']
    words = row['clean_tweet'].split()
    chunks = [' '.join(words[i:i+300]) for i in range(0, len(words), 300)]

    for chunk in chunks:
        if len(chunk.split()) >= 100:
            chunk_data.append({'label': label, 'text': chunk})

In [ ]:
chunk_df = pd.DataFrame(chunk_data)
chunk_df.head()

In [ ]:
chunk_df.index = range(1, len(chunk_df) + 1)

In [ ]:
chunk_df.head()

In [ ]:
chunk_df['text'][1]

In [ ]:
chunk_df.shape

In [ ]:
chunk_df['label'].value_counts().sort_index()

In [ ]:
chunk_df['label'].value_counts().plot(kind='bar', title='Chunks per Personality Label');

In [ ]:
chunk_df.to_csv('raw_data/mbti_chunked_clean.csv', index=True, index_label='chunk_id')